In [46]:
import swat
import pandas as pd
import numpy as np


fetch_opts = dict(to=1000000, maxrows=1000000)
dpp_opts = dict(distinctCountLimit=100000, misraGries=False)

In [47]:
server = "rdcgrd133.unx.sas.com"
port = 16271
s = swat.CAS(server, port)

In [48]:
s.loadactionset(actionset='dataPreprocess')
s.loadactionset(actionset='dataSciencePilot')
s.loadactionset(actionset='astore')
s.loadactionset(actionset='dataStep')

NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'dataPreprocess'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.024028 seconds
NOTE:       cpu time                0.117983 seconds (491.02%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  7.46M (0.00%)
NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'dataSciencePilot'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.050084 seconds
NOTE:       cpu time                0.023996 seconds (47.91%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  1.48M (0.00%)
NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'astore'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time     

[actionset]

 'dataStep'

+ Elapsed: 0.0039s, user: 0.005s, sys: 0.005s, mem: 1.44mb

In [49]:
input_tbl = "kdd98"
s.upload("U:\\data\\kdd98.csv", casout=dict(name=input_tbl, replace=True))

NOTE: Executing action 'table.loadTable'.
NOTE: Cloud Analytic Services made the uploaded file available as table KDD98 in caslib CASUSERHDFS(bigebr).
NOTE: Action 'table.loadTable' used (Total process time):
NOTE:       real time               0.576527 seconds
NOTE:       cpu time                8.500707 seconds (1474.47%)
NOTE:       data movement time      0.220051 seconds
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  699.17M (0.04%)
NOTE:       bytes moved             414.13M
NOTE: The table KDD98 has been created in caslib CASUSERHDFS(bigebr) from binary data uploaded to Cloud Analytic Services.
NOTE: Action 'table.upload' used (Total process time):
NOTE:       real time               4.649589 seconds
NOTE:       cpu time                8.817659 seconds (189.64%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  699.17M (0.

[caslib]

 'CASUSERHDFS(bigebr)'

[tableName]

 'KDD98'

[casTable]

 CASTable('KDD98', caslib='CASUSERHDFS(bigebr)')

+ Elapsed: 4.65s, user: 6.75s, sys: 2.07s, mem: 699mb

In [50]:
s.tableInfo()

NOTE: Executing action 'table.tableInfo'.
NOTE: Action 'table.tableInfo' used (Total process time):
NOTE:       real time               0.004601 seconds
NOTE:       cpu time                0.010995 seconds (238.97%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  2.40M (0.00%)


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,KDD98,95412,481,0,utf-8,2020-10-12T17:02:05-04:00,2020-10-12T17:02:05-04:00,2020-10-12T17:02:05-04:00,UTF8,1.918156e+09,...,0,0,0,,,0,bigebr,,2020-10-12T17:02:05-04:00,1.918156e+09


In [51]:
target = 'targetb'
event = "1"

# 1. Identify groups of variables using a comprehensive set of statistical metrics

In [52]:
s.dataSciencePilot.exploreData(table=input_tbl, target=target, casout=dict(name='avapt_out', replace=True));
avapt_df = s.fetch(table='avapt_out', **fetch_opts)['Fetch']

low_card_low_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 1 & MissingRated == 1')['Variable'])
low_card_medium_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 1 & MissingRated == 2')['Variable'])
low_card_high_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 1 & MissingRated == 3')['Variable'])
medium_card_low_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 2 & MissingRated == 1')['Variable'])
medium_card_medium_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 2 & MissingRated == 2')['Variable'])
medium_card_high_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 2 & MissingRated == 3')['Variable'])
high_card_low_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 3 & MissingRated == 1')['Variable'])
high_card_medium_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 3 & MissingRated == 2')['Variable'])
high_card_high_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 3 & MissingRated == 3')['Variable'])


low_skew_low_miss_interval_vars = list(avapt_df.query('SkewnessRated == 1 & MissingRated == 1')['Variable'])
low_skew_medium_miss_interval_vars = list(avapt_df.query('SkewnessRated == 1 & MissingRated == 2')['Variable'])
low_skew_high_miss_interval_vars = list(avapt_df.query('SkewnessRated == 1 & MissingRated == 3')['Variable'])
medium_skew_low_miss_interval_vars = list(avapt_df.query('SkewnessRated == 2 & MissingRated == 1')['Variable'])
medium_skew_medium_miss_interval_vars = list(avapt_df.query('SkewnessRated == 2 & MissingRated == 2')['Variable'])
medium_skew_high_miss_interval_vars = list(avapt_df.query('SkewnessRated == 2 & MissingRated == 3')['Variable'])
high_skew_low_miss_interval_vars = list(avapt_df.query('SkewnessRated == 3 & MissingRated == 1')['Variable'])
high_skew_medium_miss_interval_vars = list(avapt_df.query('SkewnessRated == 3 & MissingRated == 2')['Variable'])
high_skew_high_miss_interval_vars = list(avapt_df.query('SkewnessRated == 3 & MissingRated == 3')['Variable'])



NOTE: Executing action 'dataSciencePilot.exploreData'.
NOTE: Action 'dataSciencePilot.exploreData' used (Total process time):
NOTE:       real time               1.021758 seconds
NOTE:       cpu time                23.365448 seconds (2286.79%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  345.99M (0.02%)
NOTE:       bytes moved             175.51K
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               0.013574 seconds
NOTE:       cpu time                0.030997 seconds (228.36%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  7.72M (0.00%)


# 2. Define the interaction and non-interaction transformation pipelines

In [53]:
# example non-interaction pipelines 

pipelines = []
if len(high_card_low_miss_nominal_vars) > 0:
    p = dict(name='hclm',
              inputs = high_card_low_miss_nominal_vars,
              targets = target,
              events = event,
              impute = dict(method = 'mode'),
              mapInterval = dict(method='empBayes',
                                 args = dict(includeMissingLevel=False,
                                             rareThreshold=5)),
              discretize = dict(method = 'bucket',
                                args = dict(nBins = 5)))
    pipelines.append(p1)

if len(high_card_medium_miss_nominal_vars) > 0:
    p = dict(name='hcmm',
              inputs = high_card_medium_miss_nominal_vars,
              targets = target,
              events = event,
              mapInterval = dict(method='empBayes',
                                 args = dict(includeMissingLevel=True,
                                             rareThreshold=5)),
              discretize = dict(method = 'bucket',
                                args = dict(nBins = 5)))
    pipelines.append(p)
    
if len(high_card_high_miss_nominal_vars) > 0:
    p = dict(name='hchm',
              inputs = high_card_high_miss_nominal_vars,
              targets = target,
              events = event,
              mapInterval = dict(method='empBayes',
                                 args = dict(includeMissingLevel=True,
                                             rareThreshold=5)),
              discretize = dict(method = 'bucket',
                                args = dict(nBins = 5)))
    pipelines.append(p)

if len(low_card_high_miss_nominal_vars) > 0:
    p = dict(name='lchm',
              inputs = low_card_high_miss_nominal_vars,
              targets = target,
              events = event,
              cattrans = dict(method = 'label', 
                              args = dict(overrides=dict(binMissing=True))))
    pipelines.append(p)

if len(low_card_low_miss_nominal_vars) > 0:
    p = dict(name='lclm',
              inputs = low_card_high_miss_nominal_vars,
              targets = target,
              events = event,
              impute = dict(method = 'mode'),
              cattrans = dict(method = 'label', 
                              args = dict(overrides=dict(binMissing=True))))
    pipelines.append(p)
    
if len(high_skew_high_miss_interval_vars) > 0:
    p = dict(name='hshm',
              inputs = high_skew_high_miss_interval_vars,
              targets = target,
              events = event,
              discretize = dict(method = 'mdlp', args = dict(nBins = 5,
                                                             overrides=dict(binMissing=True))))
    pipelines.append(p)


In [54]:
# example interaction pipelines 

# e.g. among low skew, low-miss vs low card, low-miss
if len(low_skew_low_miss_interval_vars) > 0 and len(low_card_low_miss_nominal_vars):
    output_opts = dict(output=dict(noScoreTable=True, noScoreCode=True, noTable=True))
    p1 = dict(name='tr1', inputs=low_skew_low_miss_interval_vars,
              discretize=dict(method='bucket', args=dict(nBins=5)),
              **output_opts)
    p2 = dict(name='tr1', inputs=low_card_low_miss_nominal_vars,
              cattrans = dict(method='label'),
              **output_opts)
    tar_p = dict(name='tar_p', inputs=target, cattrans=dict(method='label'),
                 **output_opts)
    p = dict(name='inter1',
             interaction = dict(
                 method = 'cross',
                 synthesizer = 'nominal',
                 inputs = [p1['name'], p2['name']],
                 targets = tar_p['name']
             ),
             cattrans = dict(method='dtree', args=dict(nBins=5)))
    pipelines.append(p1)
    pipelines.append(p2)
    pipelines.append(tar_p)
    pipelines.append(p)
              
        
# e.g. among high skew, low-miss vs low card, low-miss
if len(high_skew_low_miss_interval_vars) > 0 and len(low_card_low_miss_nominal_vars):
    output_opts = dict(output=dict(noScoreTable=True, noScoreCode=True, noTable=True))
    p1 = dict(name='tr1', inputs=high_skew_low_miss_interval_vars,
              discretize=dict(method='quantile', args=dict(nBins=5)),
              **output_opts)
    p2 = dict(name='tr1', inputs=low_card_low_miss_nominal_vars,
              cattrans = dict(method='label'),
              **output_opts)
    tar_p = dict(name='tar_p', inputs=target, cattrans=dict(method='label'),
                 **output_opts)
    p = dict(name='inter2',
             interaction = dict(
                 method = 'cross',
                 synthesizer = 'nominal',
                 inputs = [p1['name'], p2['name']],
                 targets = tar_p['name']
             ),
             cattrans = dict(method='dtree', args=dict(nBins=5)))
    pipelines.append(p1)
    pipelines.append(p2)
    pipelines.append(tar_p)
    pipelines.append(p)

In [55]:
pipelines

[{'name': 'tr1',
  'inputs': ['RAMNTALL',
   'MINRAMNT',
   'MAXRAMNT',
   'LASTGIFT',
   'AVGGIFT',
   'TARGETD'],
  'discretize': {'method': 'quantile', 'args': {'nBins': 5}},
  'output': {'noScoreTable': True, 'noScoreCode': True, 'noTable': True}},
 {'name': 'hcmm',
  'inputs': ['RFA_7',
   'RFA_9',
   'RFA_11',
   'RFA_12',
   'RFA_14',
   'RFA_16',
   'RFA_18',
   'AGE',
   'NEXTDATE',
   'TIMELAG'],
  'targets': 'targetb',
  'events': '1',
  'mapInterval': {'method': 'empBayes',
   'args': {'includeMissingLevel': True, 'rareThreshold': 5}},
  'discretize': {'method': 'bucket', 'args': {'nBins': 5}}},
 {'name': 'hchm',
  'inputs': ['RFA_10',
   'RFA_13',
   'RFA_17',
   'RFA_19',
   'RFA_20',
   'RFA_21',
   'RFA_22',
   'RFA_23',
   'RFA_24'],
  'targets': 'targetb',
  'events': '1',
  'mapInterval': {'method': 'empBayes',
   'args': {'includeMissingLevel': True, 'rareThreshold': 5}},
  'discretize': {'method': 'bucket', 'args': {'nBins': 5}}},
 {'name': 'lchm',
  'inputs': ['MA

# 3. Run the FTL: Execute on the  FTE  (TRAINING)

In [56]:
s.transform(table=input_tbl,
            pipelines = pipelines,
            saveState = dict(name='scoring_model', replace=True),
            **dpp_opts)

NOTE: Executing action 'dataPreprocess.transform'.
NOTE: 187270 bytes were written to the table "scoring_model" in the caslib "CASUSERHDFS(bigebr)".
NOTE: Action 'dataPreprocess.transform' used (Total process time):
NOTE:       real time               0.797608 seconds
NOTE:       cpu time                28.766626 seconds (3606.61%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  1.49G (0.08%)
NOTE:       bytes moved             183.59K


[TransInfo]

 Transformation Requests for KDD98
 
   GivenName ActualName Interaction InputTrans1 InputTrans2 TargetTrans  \
 0      hcmm                     no                                       
 1      hchm                     no                                       
 2      lchm                     no                                       
 3      lclm                     no                                       
 4      hshm                     no                                       
 5    inter1                    yes         tr1         tr1       tar_p   
 6    inter2                    yes         tr1         tr1       tar_p   
 
    NTransVars  NEvalVars PhaseOrder ImputeMethod      MapInterval  \
 0          10          1    Map-Dis               Empirical Bayes   
 1           9          1    Map-Dis               Empirical Bayes   
 2          90          1        Cat                                 
 3          90          1    Imp-Cat         Mode                    
 4           8          1        Dis                                 
 5          36          1        Cat                                 
 6          36          1        Cat                                 
 
             DisctMethod          CatTransMethod  
 0  Equal-Width (Bucket)                          
 1  Equal-Width (Bucket)                          
 2                        Label (Sparse One-Hot)  
 3                        Label (Sparse One-Hot)  
 4                  MDLP                          
 5                                         DTree  
 6                                         DTree  

[VarTransInfo]

 Variable Transformation Information for KDD98
 
      Variable  VarIndex Interaction Transformation       Input1        Input2  \
 0    RAMNT_10       NaN          no           hshm                              
 1    RAMNT_13       NaN          no           hshm                              
 2    RAMNT_16       NaN          no           hshm                              
 3    RAMNT_17       NaN          no           hshm                              
 4    RAMNT_18       NaN          no           hshm                              
 ..        ...       ...         ...            ...          ...           ...   
 274                68.0         yes         inter2  tr1_TARGETD  tr1_MINRAMNT   
 275                69.0         yes         inter2  tr1_TARGETD  tr1_MAXRAMNT   
 276                70.0         yes         inter2  tr1_TARGETD  tr1_LASTGIFT   
 277                71.0         yes         inter2  tr1_TARGETD   tr1_AVGGIFT   
 278                72.0         yes         inter2  tr1_TARGETD   tr1_TARGETD   
 
            EvalVar      ResultVar      N  NMiss ImputedValueNominal  NBins  
 0          TARGETB  hshm_RAMNT_10  10461  84951                        6.0  
 1          TARGETB  hshm_RAMNT_13  12250  83162                        6.0  
 2          TARGETB  hshm_RAMNT_16  26994  68418                        6.0  
 3          TARGETB  hshm_RAMNT_17   9401  86011                        6.0  
 4          TARGETB  hshm_RAMNT_18  19778  75634                        6.0  
 ..             ...            ...    ...    ...                 ...    ...  
 274  tar_p_TARGETB  inter2_var_32  95412      0                        4.0  
 275  tar_p_TARGETB  inter2_var_33  95412      0                        5.0  
 276  tar_p_TARGETB  inter2_var_34  95412      0                        5.0  
 277  tar_p_TARGETB  inter2_var_35  95412      0                        5.0  
 278  tar_p_TARGETB  inter2_var_36  95412      0                        1.0  
 
 [279 rows x 12 columns]

[NomVarInfo]

 Nominal Variable Information for KDD98
 
      Variable        N    NMiss  NLevels
 0    ADATE_10  62664.0  32748.0      2.0
 1    ADATE_13  55193.0  40219.0      3.0
 2    ADATE_15  29935.0  65477.0      1.0
 3    ADATE_17  67762.0  27650.0      3.0
 4    ADATE_19  70932.0  24480.0      3.0
 ..        ...      ...      ...      ...
 149      TP

# 3. apply the scoring object to future/validation/test etc slices ... (Scoring)


In [57]:
# in this case we are simply showing how to apply it (on the same table /train tbl - replace with test/valid etc)
s.astore.score(table=input_tbl,
               rstore='scoring_model',
               # put whatever vars you want to copy here
               copyvars = [target],
               casout=dict(name='out1', replace=True))

NOTE: Executing action 'astore.score'.
NOTE: Action 'astore.score' used (Total process time):
NOTE:       real time               0.337009 seconds
NOTE:       cpu time                5.960094 seconds (1768.53%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  402.51M (0.02%)
NOTE:       bytes moved             203.85M


[OutputCasTables]

                 casLib  Name   Rows  Columns  \
 0  CASUSERHDFS(bigebr)  out1  95412      280   
 
                                          casTable  
 0  CASTable('out1', caslib='CASUSERHDFS(bigebr)')  

[Timing]

 Task Timing
 
                  Task   Seconds   Percent
 0   Loading the Store  0.008437  0.026189
 1  Creating the State  0.033186  0.103014
 2             Scoring  0.279359  0.867176
 3               Total  0.322148  1.000000

+ Elapsed: 0.337s, user: 4.87s, sys: 1.09s, mem: 403mb

In [58]:
s.tableInfo()

NOTE: Executing action 'table.tableInfo'.
NOTE: Action 'table.tableInfo' used (Total process time):
NOTE:       real time               0.004607 seconds
NOTE:       cpu time                0.009997 seconds (217.00%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  2.41M (0.00%)


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,KDD98,95412,481,0,utf-8,2020-10-12T17:02:05-04:00,2020-10-12T17:02:05-04:00,2020-10-12T17:03:12-04:00,UTF8,1.918156e+09,...,0,0,0,,,0,bigebr,,2020-10-12T17:02:05-04:00,1.918156e+09
1,AVAPT_OUT,481,42,0,utf-8,2020-10-12T17:02:07-04:00,2020-10-12T17:02:07-04:00,2020-10-12T17:02:07-04:00,UTF8,1.918156e+09,...,0,0,0,,,0,bigebr,,,NaN
2,SCORING_MODEL,1,2,0,utf-8,2020-10-12T17:02:39-04:00,2020-10-12T17:02:39-04:00,2020-10-12T17:03:12-04:00,UTF8,1.918156e+09,...,0,0,0,,,0,bigebr,,,NaN
3,OUT1,95412,280,0,utf-8,2020-10-12T17:03:12-04:00,2020-10-12T17:03:12-04:00,2020-10-12T17:03:12-04:00,UTF8,1.918156e+09,...,0,0,0,,,0,bigebr,,,NaN


In [59]:
s.close()